## Radarbanten.co.id

In [32]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [33]:
jumlah_index = 10
threads_link = []
links = []
results = []
threads = []

In [34]:
def scrape_links(page_number):
    url=f"https://www.radarbanten.co.id/category/kota-serang/page/{page_number}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "jeg_post_title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [35]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 18 links from page 5
Scraped 18 links from page 9
Scraped 18 links from page 4
Scraped 18 links from page 2
Scraped 18 links from page 10
Scraped 18 links from page 8
Scraped 18 links from page 3
Scraped 18 links from page 6
Scraped 18 links from page 7
Scraped 18 links from page 1
Total Links: 180


In [36]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "jeg_meta_author"})
            if author_elem:
                author_text = author_elem.find('a')
                author_text = author_text.text
            else:
                author_text = "Author not found"
                     
            # tanggal berita
            date_elem = soup.find('div', {"class": "jeg_meta_date"})
            if date_elem:
                date_text = date_elem.find('a')
                date_text = date_text.text
            else:
                date_text = "Date not found"
            # Category berita
            category_elements = soup.find('span', {"class": "breadcrumb_last_link"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "entry-content with-share"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [37]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [38]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  180


,title,author,category,date,content,region,link
0,"\nGawat, 2000 Keluarga di Tanara Kekurangan Air\n",Fahmi,Berita Utama,26-09-2023 12:54:19,"SERANG,RADARBANTEN.CO.ID-Sesosok mayat pria di...",banten,https://www.radarbanten.co.id/warga-temukan-ma...
1,"\nBentrok di Pasar Kutabumi, Ketua Pemuda Panc...",Mulyadi,Tangerang,25-09-2023 15:31:40,"TANGERANG, RADARBANTEN.CO.ID – DPRD Kabupaten ...",banten,https://www.radarbanten.co.id/imbas-penyeranga...
2,"\nDidaulat jadi Pembina Apel, Rano Karno Berno...",Angger Gita,Berita Utama,26-09-2023 12:59:50,"TANGERANG,RADARBANTEN.CO.ID-Dinas Pemuda dan O...",banten,https://www.radarbanten.co.id/tinggi-peminat-g...
3,\nPengelola PKL Stadion Maulana Yusuf Bantah B...,Nahrul Muhilmi,Berita Utama,17-09-2023 16:15:18,"\nSERANG,RADARBANTEN.CO.ID – Dewan Perwakilan ...",banten,https://www.radarbanten.co.id/penanganan-kemar...
4,"\nDidaulat jadi Pembina Apel, Rano Karno Berno...",Angger Gita,Berita Utama,26-09-2023 12:59:50,"TANGERANG,RADARBANTEN.CO.ID-Dinas Pemuda dan O...",banten,https://www.radarbanten.co.id/tinggi-peminat-g...
5,\nDPRD Minta Polisi Pasang Police Line di Proy...,Nahrul Muhilmi,Kota Serang,04-09-2023 21:33:46,"SERANG, RADARBANTEN.CO.ID – Subadri Ushuludin...",banten,https://www.radarbanten.co.id/wakil-walikota-s...
6,"\nImbas Penyerangan, Ketua DPRD Minta Revitali...",Mulyadi,Tangerang,25-09-2023 18:10:02,"TANGERANG, RADARBANTEN.CO.ID – Kepala Majelis ...",banten,https://www.radarbanten.co.id/bentrok-di-pasar...
7,\nWarga Temukan Mayat Pria Ngambang di Sungai ...,Nahrul Muhilmi,Kota Serang,18-09-2023 15:09:45,"SERANG, RADARBANTEN.CO.ID – Dewan Perwakilan R...",banten,https://www.radarbanten.co.id/pemkot-serang-da...
8,\nKONI Banten Bantah Staf atau Pengurusnya Dip...,Fahmi,Hukum,25-09-2023 21:19:45,"SERANG, RADARBANTEN.CO.ID – Empat pengurus KON...",banten,https://www.radarbanten.co.id/kasus-dugaan-kor...
9,"\nGawat, 2000 Keluarga di Tanara Kekurangan Air\n",Rostinah,Kota Serang,14-09-2023 15:40:38,"SERANG, RADARBANTEN.CO.ID – Tahun depan, Pempr...",banten,https://www.radarbanten.co.id/hore-tahun-depan...


# Save File 

In [39]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/radarBanten_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/radarBanten_2023-09-26_13-00-51.xlsx
